In [ ]:
!pip install datasets evaluate sentence_transformers > None
!pip install transformers accelerate

In [2]:
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_score, f1_score

# Подготовка
Данные - [Emotion](https://huggingface.co/datasets/philschmid/emotion)

In [3]:
dataset = load_dataset("emotion")
dataset["train"][100]

{'text': 'i wont let me child cry it out because i feel that loving her and lily when she was little was going to be opportunities that only lasted for those short few months',
 'label': 2}

In [4]:
model_name="bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [5]:
train_dataset = tokenized_datasets["train"].shuffle(seed=100).select(range(1000))
eval_dataset = tokenized_datasets["test"].shuffle(seed=100).select(range(1000))

<a id='trainer'></a>

# Обучение

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
metric = evaluate.load("accuracy")

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='weighted', zero_division=0)
    f1 = f1_score(labels, predictions, average='weighted', zero_division=0)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'f1': f1
    }

In [9]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", seed=100)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [10]:
result_before_tuning = trainer.evaluate(eval_dataset=eval_dataset)

In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,F1
1,No log,0.800497,0.726000,0.659983,0.671311
2,No log,0.567761,0.825000,0.814577,0.814249
3,No log,0.491459,0.854000,0.852976,0.852948


TrainOutput(global_step=375, training_loss=0.6724299723307292, metrics={'train_runtime': 390.0069, 'train_samples_per_second': 7.692, 'train_steps_per_second': 0.962, 'total_flos': 789361514496000.0, 'train_loss': 0.6724299723307292, 'epoch': 3.0})

In [12]:
result_after_tuning = trainer.evaluate()

# Оценка качества

In [13]:
print("Metrics on the evaluation dataset before tuning:")
for key, value in result_before_tuning.items():
    print(f"{key}: {value}")

print("\nMetrics on the evaluation dataset after tuning:")
for key, value in result_after_tuning.items():
    print(f"{key}: {value}")

Metrics on the evaluation dataset before tuning:
eval_loss: 1.7197792530059814
eval_accuracy: 0.362
eval_precision: 0.131044
eval_f1: 0.19242878120411158
eval_runtime: 36.2503
eval_samples_per_second: 27.586
eval_steps_per_second: 3.448

Metrics on the evaluation dataset after tuning:
eval_loss: 0.49145910143852234
eval_accuracy: 0.854
eval_precision: 0.8529758380399255
eval_f1: 0.8529481849914955
eval_runtime: 34.7584
eval_samples_per_second: 28.77
eval_steps_per_second: 3.596
epoch: 3.0


<a id='pytorch_native'></a>